In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
RANDOM_STATE = 13579 
np.random.seed(RANDOM_STATE) 

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from collections import Counter
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.neighbors import KNeighborsClassifier



In [ ]:
#Load the data
adult.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race',  'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

In [ ]:
adult = pd.read_csv(r"C:\Users\Nataqi\Repo_1\datasets\adult.data", sep=",", header=None, skipinitialspace=True)

In [4]:
adult.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [5]:
adult.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
adult.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [ ]:
#Drop missing values - "?"
missing_values = (adult == '?').sum() 
print(missing_values)

In [ ]:
#Remove any rows with missing values

def drop_missing_values(df, miss):
    
    data_dropped = df.replace(to_replace = miss, value = np.nan).dropna()
    
    return data_dropped

In [9]:
adult_dropped = drop_missing_values(adult, '?')

In [10]:
adult_dropped.shape

(30162, 15)

In [ ]:
#Assign X and y 
X = adult_dropped.drop(['income'], axis = 1)
y = adult_dropped['income']

In [12]:
(X.shape, y.shape, type(X), type(y))

((30162, 14), (30162,), pandas.core.frame.DataFrame, pandas.core.series.Series)

In [ ]:
 #One-hot encoding 
 
def one_hot_encoding(df):
 
    df_onehot = pd.get_dummies(df, drop_first = True)
    
    return df_onehot

In [14]:
X_onehot = one_hot_encoding(X)

In [15]:
X_onehot.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,39,77516,13,2174,0,40,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,50,83311,13,0,0,13,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
2,38,215646,9,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
3,53,234721,7,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
4,28,338409,13,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
#Train test split 
X_train, X_test, y_train, y_test = train_test_split(X_onehot,y, random_state = RANDOM_STATE, stratify = y, test_size = 0.2 )

In [17]:
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((24129, 96), (6033, 96), (24129,), (6033,))

In [ ]:
#Standardization - numeric columns only

def standardize(X_train, X_test, numeric):
  
    sc = StandardScaler()
    
    X_train_numeric = X_train[numeric]
    X_test_numeric = X_test[numeric]

    sc.fit(X_train_numeric)
    
  
    X_train_st = sc.transform(X_train_numeric) 
    X_test_st = sc.transform(X_test_numeric)
    
    return X_train_st, X_test_st

In [19]:
def merge(X_train, X_test, X_train_numeric, X_test_numeric, numeric):

    # to ensure that the datasets keep the Pandas DataFrame format.
    if X_train.shape == (0, 0): return pd.DataFrame([0]), pd.DataFrame([0])
    
    X_train_st_df = X_train.copy()
    X_train_st_df[numeric] = X_train_numeric
    X_test_st_df = X_test.copy()
    X_test_st_df[numeric] = X_test_numeric
    
    return X_train_st_df, X_test_st_df

In [20]:
numeric = (X_train.select_dtypes(include='number').columns).to_list() #find numeric columns

In [21]:
X_train_numeric, X_test_numeric = standardize(X_train, X_test, numeric)

In [22]:
X_train_numeric.mean(axis=0)  #Check the mean and standard deviation values of the standardized dataset 

array([ 1.82575530e-16, -3.29813861e-17,  8.42203251e-17, -2.82697595e-17,
       -4.47604525e-17,  1.41348797e-17])

In [23]:
X_train_numeric.mean(axis=0)

array([ 1.82575530e-16, -3.29813861e-17,  8.42203251e-17, -2.82697595e-17,
       -4.47604525e-17,  1.41348797e-17])

In [24]:
X_train_numeric.std(axis=0)

array([1., 1., 1., 1., 1., 1.])

In [25]:
X_train_st, X_test_st = merge(X_train, X_test, X_train_numeric, X_test_numeric, numeric) #scikit-learn's StandardScaler does not return DataFrame. Recover DataFrame and categorical features.

In [26]:
X_train_st.describe()


,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,2.412900e+04,2.412900e+04,2.412900e+04,2.412900e+04,2.412900e+04,2.412900e+04
mean,1.825755e-16,-3.298139e-17,8.422033e-17,-2.826976e-17,-4.476045e-17,1.413488e-17
std,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00,1.000021e+00
min,-1.637860e+00,-1.667683e+00,-3.574146e+00,-1.471472e-01,-2.153033e-01,-3.333406e+00
25%,-7.991874e-01,-6.804452e-01,-4.392511e-01,-1.471472e-01,-2.153033e-01,-7.805663e-02
50%,-1.130010e-01,-1.067059e-01,-4.738916e-02,-1.471472e-01,-2.153033e-01,-7.805663e-02
75%,6.494285e-01,4.495231e-01,1.128197e+00,-1.471472e-01,-2.153033e-01,3.392959e-01
max,3.927875e+00,1.229248e+01,2.303782e+00,1.373371e+01,1.070320e+01,4.846703e+00


In [ ]:
#Classification - Random forest

rf = RandomForestClassifier(n_estimators=100, random_state = RANDOM_STATE)

rf_cross_val_score = np.mean(cross_val_score(rf, X_onehot, y, cv = 5))

grid_dict = {'max_depth':[3,4,5,6], 'min_samples_split':[3, 5, 7]} 

In [30]:
gs = GridSearchCV(rf, param_grid = grid_dict, cv=5)
gs.fit(X_train_st, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=13579),
             param_grid={'max_depth': [3, 4, 5, 6],
                         'min_samples_split': [3, 5, 7]})

In [31]:
rf_best_classifier = gs.best_estimator_  #best classifier instance
rf_best_classifier

RandomForestClassifier(max_depth=6, min_samples_split=5, random_state=13579)

In [ ]:
#SVM with diverse kernels

svc = SVC()
grid_dict = [{'kernel':['linear', 'poly', 'rbf']}, {'C':[1, 10, 100]}]
gs = GridSearchCV(svc, param_grid = grid_dict, cv=5)
gs.fit(X_train_st, y_train)
svm_best_classifier = gs.best_estimator_
svm_best_classifier

SVC()

In [33]:
svm_gs_score = gs.best_score_ # best model from grid search
svm_gs_score

0.8537860767131406

  - 8-1. Entropy, Gini index, and information gain 
  - 8-2. Tree implementation

- Task 8-1: Create a Gini index function.
  - The Gini Index is calculated by subtracting the sum of the squared probabilities of each class from one.
  - You should double-check the lecture slides and the examples below to ensure you made a correct function.
  - You can use `collections.Counter()` to count labels of the dataset.

In [ ]:
#Decision tree 
playgolf = pd.read_csv(r'C:\Users\Nataqi\учеба\Stockholm university\DAMI HT23\3\datasets\playgolf.csv')

In [35]:
playgolf.columns

Index(['Outlook', 'Temp', 'Humidity', 'Windy', 'Play Golf'], dtype='object')

In [36]:
playgolf.head()

,Outlook,Temp,Humidity,Windy,Play Golf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes


In [ ]:
def gini(dataset):  
     
    impurity = 0
    x = Counter(dataset)
    sum_values = sum(x.values())
    
    for value in x.values():
        probability = value/sum_values
        pred_impurity = probability*(1-probability)
        impurity += pred_impurity
        
    return impurity

'\n    A function that calculates the Gini index of a given list.\n    \n    Input\n     - dataset: a list of labels.\n    Output\n     - impurity: The Gini index of the list.\n    \n    '

In [38]:
gini([0,0,1,1])

0.5

In [39]:
gini_score = gini(playgolf["Windy"])

In [40]:
gini_score

0.4897959183673469

In [ ]:
def entropy(dataset):
    
    entropy = 0 
    n = Counter(dataset)
    sum_values = sum(n.values())
    
    for value in n.values():
        probability = value/sum_values
        pred_entropy = np.log2(1/probability)
        pred_entropy_2 = pred_entropy*probability
        entropy += pred_entropy_2
        
    return entropy


'\n    Input\n     - dataset: a list of labels.\n    Output\n     - impurity: entropy value of the list.\n    \n    '

In [44]:
entropy_score = entropy(playgolf['Temp']) 
entropy_score

1.5566567074628228

In [ ]:
def information_gain(labels_start, labels_split):
    gini_subset_sum = 0
    
    for i in range(len(labels_split)):
        gini_subset_sum += gini(labels_split[i])*(len(labels_split[i])/len(labels_start))
        info_gain = gini(labels_start) - gini_subset_sum
        
    return info_gain  

In [49]:
def split(X, y, attr):
    split_attrs = []
    split_labels = []
    
    for val in X[attr].unique():
        attr_subset = []
        label_subset = []
        
        for idx, row in X.iterrows():
            
            if row[attr] == val:
                attr_subset.append(row)
                label_subset.append(y[idx])
                
        split_attrs.append(pd.DataFrame(attr_subset).reset_index(drop= True))
        split_labels.append(label_subset)
        

        
    return split_attrs, split_labels

In [50]:
split(playgolf.drop('Play Golf', axis=1), playgolf['Play Golf'], 'Windy')

([    Outlook  Temp Humidity  Windy
  0     Rainy   Hot     High  False
  1  Overcast   Hot     High  False
  2     Sunny  Mild     High  False
  3     Sunny  Cool   Normal  False
  4     Rainy  Mild     High  False
  5     Rainy  Cool   Normal  False
  6     Sunny  Mild   Normal  False
  7  Overcast   Hot   Normal  False,
      Outlook  Temp Humidity  Windy
  0     Rainy   Hot     High   True
  1     Sunny  Cool   Normal   True
  2  Overcast  Cool   Normal   True
  3     Rainy  Mild   Normal   True
  4  Overcast  Mild     High   True
  5     Sunny  Mild     High   True],
 [['No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'Yes', 'Yes'],
  ['No', 'No', 'Yes', 'Yes', 'Yes', 'No']])

In [51]:
playgolf['Windy']

0     False
1      True
2     False
3     False
4     False
5      True
6      True
7     False
8     False
9     False
10     True
11     True
12    False
13     True
Name: Windy, dtype: bool

In [ ]:
def select_attributes(X, strategy):
    if type(strategy) == int:
        num_attr = strategy
    elif strategy == 'sqrt':
        num_attr = int(np.sqrt(len(X.columns))) 
    elif strategy == 'max':
        num_attr = len(X.columns)
        
    attributes = (np.random.choice(X.columns, num_attr, replace = True)).tolist()

    return attributes

In [53]:
select_attributes(playgolf.drop('Play Golf', axis=1), 2)

['Windy', 'Temp']

In [54]:
select_attributes(playgolf.drop('Play Golf', axis=1), "max")

['Windy', 'Outlook', 'Windy', 'Humidity']

In [ ]:
def check_info_gain_per_attribute(X, y, attributes):
    best_info_gain = 0
    best_attr = None
    
    for i in attributes:
        split_attrs, split_labels = split(X, y, i)
        info_gain = information_gain(y, split_labels) 
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_attr = i
            
    return best_attr, best_info_gain

In [56]:
best_attribute, best_info_gain = check_info_gain_per_attribute(playgolf.drop('Play Golf', axis=1), playgolf["Play Golf"], ["Windy", "Outlook"])
print(best_attribute)

Outlook


In [ ]:
def best_split(X, y, strategy):
    attributes = select_attributes(X, strategy)
    best_attr, best_info_gain = check_info_gain_per_attribute(X, y, attributes)
    
    return best_attr, best_info_gain

In [58]:
np.random.seed(RANDOM_STATE)
best_attr_playgolf, best_gain_playgolf = best_split(playgolf.drop('Play Golf', axis=1), playgolf['Play Golf'] , 'sqrt')

In [59]:
best_attr_playgolf, best_gain_playgolf

('Outlook', 0.11632653061224485)

In [ ]:
def build(X, y, strategy, max_depth = 5, min_samples_leaf = 5, tol=0.00001, _depth = 0):
    
    if _depth > max_depth or len(X) < min_samples_leaf:
        return {"type": "leaf", "majority": max(Counter(y))}
    
    best_attr, best_info_gain = best_split(X, y, strategy) 
    
    if best_info_gain < tol:
        return {"type": "leaf", "gain": best_info_gain, "majority": max(Counter(y))}
    
    split_attrs, split_labels = split(X, y, best_attr)
    
    branches = []      
    for i in range(len(split_attrs)):
        a = build(split_attrs[i],split_labels[i], strategy, max_depth, min_samples_leaf, tol, _depth+1)
        branches.append(a)      

    return {
            "type": "node",
            "best_feature": best_attr,
            "branches": branches,
            "value": best_info_gain 
    }

In [61]:
np.random.seed(RANDOM_STATE)

single_tree = build(playgolf.drop('Play Golf', axis=1), playgolf['Play Golf'], 'sqrt', max_depth = 3, min_samples_leaf = 2)

In [62]:
single_tree

{'type': 'node',
 'best_feature': 'Outlook',
 'branches': [{'type': 'leaf', 'gain': 0, 'majority': 'Yes'},
  {'type': 'leaf', 'gain': 0, 'majority': 'Yes'},
  {'type': 'node',
   'best_feature': 'Humidity',
   'branches': [{'type': 'node',
     'best_feature': 'Windy',
     'branches': [{'type': 'leaf', 'majority': 'Yes'},
      {'type': 'leaf', 'majority': 'No'}],
     'value': 0.5},
    {'type': 'node',
     'best_feature': 'Temp',
     'branches': [{'type': 'node',
       'best_feature': 'Windy',
       'branches': [{'type': 'leaf', 'majority': 'Yes'},
        {'type': 'leaf', 'majority': 'No'}],
       'value': 0.5},
      {'type': 'leaf', 'majority': 'Yes'}],
     'value': 0.11111111111111116}],
   'value': 0.013333333333333308}],
 'value': 0.11632653061224485}

In [ ]:
#Evaluation
y_train_numeric = pd.Series(y_train.replace(to_replace = ['<=50K', '>50K'], value = [0,1])) 
y_test_numeric = pd.Series(y_test.replace(to_replace = ['<=50K', '>50K'], value = [0,1]))

In [64]:
y_train_numeric.unique(), y_test_numeric.unique()

(array([1, 0], dtype=int64), array([0, 1], dtype=int64))

In [65]:
svc = SVC(kernel = 'poly')
svc.fit(X_train_st,y_train_numeric)
y_pred_svc = svc.predict(X_test_st)

precision_score_svc = precision_score(y_test_numeric, y_pred_svc, average='macro')  
recall_score_svc = recall_score(y_test_numeric, y_pred_svc, average='macro')
f1_score_svc = f1_score(y_test_numeric, y_pred_svc, average='macro')

In [66]:
precision_score_svc, recall_score_svc, f1_score_svc

(0.8110794247077231, 0.7462773096476087, 0.7695547365499339)

In [ ]:
#AUC/AUPRC
f_10 = RandomForestClassifier(random_state = RANDOM_STATE) 
rf_10.fit(X_train_st, y_train_numeric)
y_pred_rf = rf_10.predict(X_test_st)

In [68]:
accuracy_rf = accuracy_score(y_test_numeric, y_pred_rf)

In [69]:
auc_rf = roc_auc_score(y_test_numeric, rf_10.predict_proba(X_test_st)[:,1], average = 'weighted')
auprc_rf = average_precision_score(y_test_numeric,rf_10.predict_proba(X_test_st)[:,1], average = 'weighted')

In [70]:
(accuracy_rf, auc_rf, auprc_rf)

(0.8483341621084037, 0.9001796618706874, 0.7680117192095133)

In [ ]:
#KNN, GridSearchCV, Average Precision, AUPRC
knn = KNeighborsClassifier()

grid_dict_knn = {'n_neighbors': [1,2,3,4,5,6,7,8,9,10]}
gs_AUPRC = GridSearchCV(knn, param_grid = grid_dict_knn, cv = 3, scoring = 'average_precision')
gs_average_precision = GridSearchCV(knn, param_grid = grid_dict_knn, cv = 3, scoring = 'precision')
gs_AUPRC.fit(X_train_st, y_train_numeric)
gs_average_precision.fit(X_train_st, y_train_numeric)

auprc_best_classifier = gs_AUPRC.best_estimator_
precision_best_classifier = gs_average_precision.best_estimator_

In [72]:
(auprc_best_classifier, precision_best_classifier)

(KNeighborsClassifier(n_neighbors=10), KNeighborsClassifier(n_neighbors=10))

In [ ]:
#deployment
diabetes = pd.read_csv(r'C:\Users\Nataqi\учеба\Stockholm university\DAMI HT23\3\datasets\diabetes.csv')

In [82]:
X = diabetes.drop('Outcome', axis = 1) 
y = diabetes['Outcome']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = RANDOM_STATE, stratify = y, test_size = 0.1)

In [84]:
import dash

In [85]:
svc = SVC()
grid_dict = [{'kernel':['poly'],'degree' : [3, 4, 5]},{'kernel':['linear', 'rbf'],'C' : [5, 10, 100]}]

gs_svc_2 = GridSearchCV(svc, param_grid = grid_dict, cv=3, scoring = 'average_precision')
trained_model_svc_2 = gs_svc_2.fit(X_train, y_train)

svm_best_classifier_dash = gs_svc_2.best_estimator_

with open('model_diabetes.pickle', 'wb') as f:
    pickle.dump(svm_best_classifier_dash, f)
